### 1. Libraries

In [54]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression

### 2. Helper Functions

In [55]:
def compute_x_error(k, b, y_measured, x_true):
    """
    Compute the error in x after inverting a linear model y = kx + b.
    
    Parameters:
    - k (float): Slope of the linear model
    - b (float): Intercept of the linear model
    - y_measured (float or np.ndarray): Measured output y
    - x_true (float or np.ndarray): Ground truth input x

    Returns:
    - x_error (float or np.ndarray): Absolute error (x_true - x_estimated)
    - x_estimated (float or np.ndarray): Estimated x from inverting the model
    """
    # Invert the model: y = kx + b → x = (y - b) / k
    x_estimated = (y_measured - b) / k

    # Compute error
    x_error = x_true - x_estimated

    return x_error, x_estimated
    
def fit_linear_model(x, y):
    """
    Fit a simple linear regression model (y = kx + b).

    Parameters:
    - x (np.ndarray): Input data, shape (n_samples, 1)
    - y (np.ndarray): Output data, shape (n_samples, 1)

    Returns:
    - y_pred (np.ndarray): Predicted y values from the model
    - k (float): Slope (coefficient) of the fitted line
    - b (float): Intercept of the fitted line
    - mae (float): Mean Absolute Error
    - rmse (float): Root Mean Squared Error
    """
    model = LinearRegression(fit_intercept=True)
    model.fit(x, y)

    y_pred = model.predict(x)

    k = model.coef_[0][0]
    b = model.intercept_[0]
    mae = mean_absolute_error(y, y_pred)
    rmse = np.sqrt(mean_squared_error(y, y_pred))

    return y_pred, k, b, mae, rmse

### 3. Import Data

In [56]:
# Define dataset folders with meaningful keys
dataset_paths = {
    "J68894_420R": "data/J68894_420R",
    "J3929_450R": "data/J3929_450R",
    "F21329": "data/F21329",
    "J0636_446R": "data/J0636_446R",
    "F15021": "data/F15021",
    "J64906_406R": "data/J64906_406R",
    "J66248_407R": "data/J66248_407R",
    "J532296_502R": "data/J532296_502R"
}

# Store loaded data in dictionaries
signal_params = {}
signals = {}

# Load each dataset
for key, folder in dataset_paths.items():
    signal_params[key] = pd.read_csv(f"{folder}/sigParam.csv")
    signals[key] = pd.read_csv(f"{folder}/signal.csv")

# Data outline
for sensor in signal_params.keys():
    unique_inds = signal_params[sensor]['ind'].nunique()
    print(f"Sensor: {sensor} → Measured signals: {unique_inds}") 


Sensor: J68894_420R → Measured signals: 200
Sensor: J3929_450R → Measured signals: 200
Sensor: F21329 → Measured signals: 200
Sensor: J0636_446R → Measured signals: 200
Sensor: F15021 → Measured signals: 200
Sensor: J64906_406R → Measured signals: 200
Sensor: J66248_407R → Measured signals: 200
Sensor: J532296_502R → Measured signals: 200


### 4. Combine & Filter Data

In [57]:
# List to store processed data for all sensors
all_sensors_data = []

# Loop through each sensor name
for sensor_name in signal_params:
    param_df = signal_params[sensor_name]
    signals_df = signals[sensor_name]

    # Select and rename relevant columns
    params = (
        param_df[['ind', 'conc', 'temp']]
        .rename(columns={'ind': 'sig'})
        .copy()
    )

    # Remove entries with zero concentration
    params = params[params['conc'] != 0]

    # Add sensor name as a column
    params['sensor'] = sensor_name

    # Compute ymax and tmax for each signal
    def get_ymax_tmax(sig_id):
        signal = signals_df[signals_df['sig'] == sig_id]
        if signal.empty:
            return pd.Series({'ymax': None, 'tmax': None})
        idx_max = signal['y'].idxmax()
        return pd.Series({
            'ymax': signal.at[idx_max, 'y'],
            'tmax': signal.at[idx_max, 't']
        })

    params[['ymax', 'tmax']] = params['sig'].apply(get_ymax_tmax)

    # Append to the combined list
    all_sensors_data.append(params)

# Combine all sensor data into one DataFrame
data = pd.concat(all_sensors_data, ignore_index=True)

# Date review
print(f"The dataset contains {len(data)} signals, described by the parameters:\n{list(data.columns)}\nacross {len(dataset_paths)} different sensors.")

The dataset contains 1200 signals, described by the parameters:
['sig', 'conc', 'temp', 'sensor', 'ymax', 'tmax']
across 8 different sensors.


### 5. Compute Integrals

In [58]:
def compute_integrals(sensor_key, signal_id, t_max):
    """
    Compute integrals and decay rate for given sensor and signal.

    Parameters:
    - sensor_key (str): Sensor name key matching dataset_paths keys
    - signal_id (int): Signal identifier
    - t_max (float): Prediction time horizon
    
    Returns:
    - integral_measured, integral_predicted, decay_rate
    """
    param_df = signal_params[sensor_key]
    signal_df = signals[sensor_key]

    # Filter signal rows for given signal id and valid 'y' to avoid NaNs
    signal_data = signal_df[(signal_df['sig'] == signal_id) & signal_df['y'].notna()]

    t = signal_data['t'].to_numpy()
    y = signal_data['y'].to_numpy()

    # Keep only positive y values
    positive_mask = y > 0
    t = t[positive_mask]
    y = y[positive_mask]

    if len(y) == 0:
        return np.nan, np.nan, np.nan

    # Find time at peak
    peak_idx = np.argmax(y)
    t_peak = t[peak_idx]

    # Keep only data after the peak
    post_peak_mask = t >= t_peak
    t_fit = t[post_peak_mask]
    y_fit = y[post_peak_mask]

    # Further restrict to y > 1 to avoid log issues
    valid_mask = y_fit > 1
    t_fit = t_fit[valid_mask]
    y_fit = y_fit[valid_mask]

    if len(y_fit) < 2:
        return np.nan, np.nan, np.nan

    log_y_fit = np.log(y_fit)
    model = LinearRegression(fit_intercept=True)
    model.fit(t_fit.reshape(-1, 1), log_y_fit)
    decay_rate = model.coef_[0]

    # Predict future decay and compute integrals
    t_pred = np.arange(t[-1], t_max + 0.002, 0.002)
    log_y_pred = model.predict(t_pred.reshape(-1, 1))
    y_pred = np.exp(log_y_pred)

    integral_measured = np.trapezoid(y, t)
    integral_predicted = np.trapezoid(y_pred, t_pred)

    return integral_measured, integral_predicted, decay_rate


# -------------------
# Apply to all sensors
# -------------------
for sensor_key in dataset_paths.keys():
    param_df = signal_params[sensor_key]
    unique_signals = param_df['ind'].unique()

    for signal_id in unique_signals:
        # Compute integrals and decay
        int_meas, int_pred, decay = compute_integrals(sensor_key, signal_id, t_max=180)

        # Update main DataFrame
        mask = (data['sensor'] == sensor_key) & (data['sig'] == signal_id)
        print()
        data.loc[mask, 'decay'] = decay
        data.loc[mask, 'int0'] = int_meas
        data.loc[mask, 'intp'] = int_pred
        data.loc[mask, 'intnt'] = int_meas + int_pred if not np.isnan(int_meas) and not np.isnan(int_pred) else np.nan

    # Fit linear model for this sensor
    sensor_mask = data['sensor'] == sensor_key
    sensor_data = data[sensor_mask].dropna(subset=['conc', 'intnt'])

    if not sensor_data.empty:
        conc = sensor_data['conc'].to_numpy().reshape(-1, 1)
        intnt = sensor_data['intnt'].to_numpy().reshape(-1, 1)

        intnt_calibr, k, b, mae, rmse = fit_linear_model(conc, intnt)

        # Write scalar values, and array for intnt_calibr
        data.loc[sensor_mask, 'k'] = k
        data.loc[sensor_mask, 'b'] = b
        data.loc[sensor_mask, 'mae'] = mae
        data.loc[sensor_mask, 'rmse'] = rmse
        data.loc[sensor_mask, 'intnt_calibr'] = intnt_calibr.flatten()


C:\Users\alexp\AppData\Local\Temp\ipykernel_17732\688285484.py:55: RuntimeWarning: overflow encountered in exp
  y_pred = np.exp(log_y_pred)
c:\Users\alexp\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_function_base_impl.py:5102: RuntimeWarning: overflow encountered in add
  ret = (d * (y[tuple(slice1)] + y[tuple(slice2)]) / 2.0).sum(axis)


### 6. Compute Concentration Errors

In [59]:
for sensor_key in dataset_paths.keys():
    param_df = signal_params[sensor_key]

    # Get all unique signal IDs for this sensor
    unique_signals = param_df['ind'].unique()

    for signal_id in unique_signals:
        # Create mask for current sensor and signal
        mask = (data['sensor'] == sensor_key) & (data['sig'] == signal_id)

        # Extract data using the mask
        conc_true = data.loc[mask, 'conc'].values
        intnt = data.loc[mask, 'intnt'].values
        k = data.loc[mask, 'k'].values
        b = data.loc[mask, 'b'].values

        # Skip if any value is missing
        if len(conc_true) == 0 or len(intnt) == 0 or len(k) == 0 or len(b) == 0:
            continue

        # Get scalar values
        conc_true_val = conc_true[0]
        intnt_val = intnt[0]
        k_val = k[0]
        b_val = b[0]

        # Compute error and estimate using the updated function
        conc_err, conc_measured = compute_x_error(k_val, b_val, intnt_val, conc_true_val)

        # Compute relative error (optional)
        conc_rel_err = conc_err / conc_true_val if conc_true_val != 0 else np.nan

        # Store the results
        data.loc[mask, 'conc_err'] = conc_err
        data.loc[mask, 'conc_rel_err'] = conc_rel_err
        data.loc[mask, 'conc_measured'] = conc_measured


### 7. Save Data

In [60]:
data.to_csv('data/data.csv', index=False)